In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from clean.harahan_csd import clean
from clean.harahan_pd import clean as clean_pd

In [2]:
pprr = clean()
pprr

,employee_id,annual_salary,employment_status,rank,first_name,middle_name,middle_initial,last_name,left_date,hire_year,hire_month,hire_day,data_production_year,agency
0,2146,66044.16,inactive,captain,manuel,j,j,adams,6/29/20,2002,1,2,2020,Harahan CSD
1,2308,40906.32,inactive,police officer,chase,m,m,bagley,10/16/20,2019,2,24,2020,Harahan CSD
2,2335,40513.20,active,police officer,michael,,,barnett,NaN,2020,2,17,2020,Harahan CSD
3,2270,16640.00,inactive,i/t coordinator,gregory,j,j,binder,NaN,2013,12,4,2020,Harahan CSD
4,2268,50000.08,inactive,assistant chief of police,aaron,j,j,blackwell,5/9/14,2013,11,3,2020,Harahan CSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2278,42681.60,inactive,secretary - police chief,tracy,m,m,whittle,5/29/19,2014,6,2,2020,Harahan CSD
122,2287,37850.90,inactive,police sergeant,michael,p,p,wright,12/19/18,2015,8,2,2020,Harahan CSD
123,2205,30500.08,inactive,police officer,andrew,d,d,yeadon,1/29/09,2008,4,2,2020,Harahan CSD
124,2199,30500.08,inactive,police officer,robert,m,m,yeadon,8/2/08,2007,7,6,2020,Harahan CSD


In [3]:
pprr.employee_id.unique()

array([2146, 2308, 2335, 2270, 2268, 2321, 2282, 2300, 2289, 2292, 2299,
       2294, 2302, 2209, 2258, 2204, 2207, 2296, 2136, 2073, 2140, 2150,
       2260, 2172, 2345, 2264, 2304, 2295, 2284, 2183, 2254, 2347, 2202,
       2230, 2166, 2212, 2246, 2305, 2286, 2208, 2220, 2227, 2226, 2221,
       2319, 2211, 2224, 2297, 2331, 2216, 2343, 2242, 2201, 2266, 2291,
       2315, 2203, 2276, 2350, 2214, 2206, 2193, 2119, 2317, 2174, 2248,
       2190, 2323, 2250, 2312, 2311, 2223, 2280, 2303, 2327, 2234, 2180,
       2244, 2238, 2274, 2210, 2232, 2310, 2044, 2219, 2252, 2313, 2092,
       2047, 2225, 2290, 2218, 2170, 2288, 2329, 2333, 2240, 2195, 2325,
       2217, 2306, 2161, 2351, 2337, 2156, 2236, 2339, 2245, 2298, 2192,
       2141, 2147, 2349, 2293, 2341, 2262, 2213, 2197, 1082, 2301, 2278,
       2287, 2205, 2199, 2256])

In [4]:
pdpprr = clean_pd()
pdpprr

,last_name,first_name,badge_no,rank_desc,rank_year,rank_month,rank_day
0,walker,tim,1,,,,
1,bagley,chase,187,,,,
2,bloomer,jordon,195,sergeant,2019,10,14
3,crovetto,eric,117,sergeant,2013,3,14
4,darwin,david,106,sergeant,2019,5,15
5,deacon,drew,129,sergeant,2020,3,2
6,dunning,john,201,,,,
7,evans,benjamin,194,,,,
8,hadden,susan,189,,,,
9,hammond,chris,163,,,,
